In [14]:
%%bash
cd ../..
pip install -U pip
pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///mnt/c/Users/Tyler/reinforcement-learning-gym
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: reinforce
    Found existing installation: reinforce 0.0.1
    Uninstalling reinforce-0.0.1:
      Successfully uninstalled reinforce-0.0.1
  Running setup.py develop for reinforce


# Single and Multi-Armed Bandits Lab

In [15]:
from reinforce.scenarios.bandit import Bandit
from reinforce.agents.agent import Agent
from reinforce.policies.policy import GreedyPolicy, EpsilonGreedyPolicy, RandomPolicy
from reinforce.environments.environment import Environment
from reinforce.utils.utils import plot_env_result

ImportError: cannot import name 'RandomPolicy' from 'reinforce.policies.policy' (C:\Users\Tyler\reinforcement-learning-gym\reinforce\policies\policy.py)

In [ ]:
k_arms = 10
steps = 2000
runs = 100

scenario = Bandit(arms=k_arms, mean=0, std=1)
results = []

# Test epsilon-greedy
eps = [0.01, 0.1, 0.2]
for epsilon in eps:
    agent = Agent(num_actions=k_arms, policy=EpsilonGreedyPolicy(epsilon=epsilon))
    environment = Environment(scenario=scenario, agent=agent, steps=steps, iterations=runs)
    environment.summarize_test()
    results.append(environment.results)

In [ ]:
plot_env_result(results)